In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

def calculate_metrics(df):
    df = df[df['label'] != -1]
    return {
        'Accuracy': accuracy_score(df['label'], df['pred']),
        'Precision': precision_score(df['label'], df['pred'], average='weighted', zero_division=0),
        'Recall': recall_score(df['label'], df['pred'], average='weighted', zero_division=0),
        'F1': f1_score(df['label'], df['pred'], average='weighted', zero_division=0)
    }

datasets = {
    'dirty_DBLP_ACM': pd.read_csv('llama3_predictions/dirty_DBLP-ACM.csv'),
    'dirty_iTunes_Amazon': pd.read_csv('llama3_predictions/dirty_iTunes-Amazon.csv'),
    'dirty_Walmart_Amazon': pd.read_csv('llama3_predictions/dirty_Walmart-Amazon.csv'),
    'structured_DBLP_ACM': pd.read_csv('llama3_predictions/structured_DBLP-ACM.csv'),
    'structured_iTunes_Amazon': pd.read_csv('llama3_predictions/structured_iTunes-Amazon.csv'),
    'structured_Walmart_Amazon': pd.read_csv('llama3_predictions/structured_Walmart-Amazon.csv'),
    'structured_Beer': pd.read_csv('llama3_predictions/structured_Beer.csv'),
    'structured_Fodors_Zagats': pd.read_csv('llama3_predictions/structured_Fodors-Zagats.csv'),
    'structured_Amazon_Google': pd.read_csv('llama3_predictions/structured_Amazon-Google.csv'),
    'textual_Abt_Buy': pd.read_csv('llama3_predictions/textual_Abt-Buy.csv')
}

results = {name: calculate_metrics(df) for name, df in datasets.items()}

df_all = pd.DataFrame(results).T
df_all.style.background_gradient(cmap='RdYlGn')

/Users/carlhelin/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,Accuracy,Precision,Recall,F1
dirty_DBLP_ACM,0.481860,0.800409,0.481860,0.526598
dirty_iTunes_Amazon,0.373853,0.711752,0.373853,0.348094
dirty_Walmart_Amazon,0.635310,0.900406,0.635310,0.710637
structured_DBLP_ACM,0.437424,0.826858,0.437424,0.469969
structured_iTunes_Amazon,0.403670,0.699751,0.403670,0.397946
structured_Walmart_Amazon,0.565886,0.905998,0.565886,0.650477
structured_Beer,0.708791,0.894891,0.708791,0.749321
structured_Fodors_Zagats,0.748677,0.891771,0.748677,0.791419
structured_Amazon_Google,0.679241,0.879788,0.679241,0.742128
textual_Abt_Buy,0.715423,0.904061,0.715423,0.769204


In [3]:
def calculate_metrics(df):
    df = df[df['label'] != -1]
    metrics = {}
    for name, group in df.groupby(['general_or_domain', 'simple_or_complex']):
        metrics[name] = {
            'Accuracy': accuracy_score(group['label'], group['pred']),
            'Precision': precision_score(group['label'], group['pred'], average='weighted', zero_division=0),
            'Recall': recall_score(group['label'], group['pred'], average='weighted', zero_division=0),
            'F1': f1_score(group['label'], group['pred'], average='weighted', zero_division=0)
        }
    return metrics

results = {}
for name, df in datasets.items():
    results.update({(name,) + key: value for key, value in calculate_metrics(df).items()})

df_all = pd.DataFrame(results).T
df_all.index.names = ['Dataset', 'GeneralOrDomain', 'SimpleOrComplex']
df_all.style.background_gradient(subset=['Accuracy', 'Precision', 'Recall', 'F1'], cmap='RdYlGn')

In [4]:
def calculate_metrics(df):
    df = df[df['label'] != -1]
    metrics = {}
    for category in ['general_or_domain', 'simple_or_complex']:
        for name, group in df.groupby(category):
            metrics[name] = {
                'F1': f1_score(group['label'], group['pred'], average='weighted', zero_division=0)
            }
    return metrics

results = []
for name, df in datasets.items():
    metrics = calculate_metrics(df)
    results.append((name, 
                    metrics.get('domain', {}).get('F1', None), 
                    metrics.get('general', {}).get('F1', None),
                    metrics.get('simple', {}).get('F1', None),
                    metrics.get('complex', {}).get('F1', None)))

df_all = pd.DataFrame(results, columns=['Dataset', 'Domain F1', 'General F1', 'Simple F1', 'Complex F1'])
df_all.set_index('Dataset', inplace=True)
df_all.style.background_gradient(subset=['Domain F1', 'General F1', 'Simple F1', 'Complex F1'], cmap='RdYlGn')



,Domain F1,General F1,Simple F1,Complex F1
Dataset,,,,
dirty_DBLP_ACM,0.604904,0.438771,0.408071,0.631277
dirty_iTunes_Amazon,0.463546,0.211055,0.388684,0.302084
dirty_Walmart_Amazon,0.796029,0.615483,0.679800,0.740776
structured_DBLP_ACM,0.592196,0.322768,0.326133,0.592276
structured_iTunes_Amazon,0.567620,0.180515,0.367076,0.428950
structured_Walmart_Amazon,0.791829,0.476231,0.632057,0.668557
structured_Beer,0.744591,0.754051,0.509663,0.958205
structured_Fodors_Zagats,0.839533,0.743070,0.611793,0.963319
structured_Amazon_Google,0.751703,0.732426,0.730564,0.753485


In [8]:
def calculate_metrics(df):
    df = df[df['label'] != -1]
    metrics = {}
    for (general_or_domain, simple_or_complex), group in df.groupby(['general_or_domain', 'simple_or_complex']):
        name = f'{general_or_domain}_{simple_or_complex}'
        metrics[name] = {
            'F1': f1_score(group['label'], group['pred'], average='weighted', zero_division=0)
        }
    return metrics

results = []
for name, df in datasets.items():
    metrics = calculate_metrics(df)
    results.append((name, 
                    metrics.get('domain_simple', {}).get('F1', None), 
                    metrics.get('domain_complex', {}).get('F1', None),
                    metrics.get('general_simple', {}).get('F1', None),
                    metrics.get('general_complex', {}).get('F1', None)))

df_all = pd.DataFrame(results, columns=['Dataset', 'Domain Simple F1', 'Domain Complex F1', 'General Simple F1', 'General Complex F1'])
df_all['Mean F1'] = df_all[['Domain Simple F1', 'Domain Complex F1', 'General Simple F1', 'General Complex F1']].mean(axis=1)
df_all.set_index('Dataset', inplace=True)
df_all.style.background_gradient(subset=['Domain Simple F1', 'Domain Complex F1', 'General Simple F1', 'General Complex F1', 'Mean F1'], cmap='RdYlGn', axis=1)

,Domain Simple F1,Domain Complex F1,General Simple F1,General Complex F1,Mean F1
Dataset,,,,,
dirty_DBLP_ACM,0.426490,0.751518,0.389502,0.486814,0.513581
dirty_iTunes_Amazon,0.479110,0.446947,0.287961,0.117210,0.332807
dirty_Walmart_Amazon,0.819235,0.771703,0.509201,0.708998,0.702284
structured_DBLP_ACM,0.357289,0.777154,0.293390,0.351407,0.444810
structured_iTunes_Amazon,0.504264,0.628238,0.205976,0.150067,0.372136
structured_Walmart_Amazon,0.830575,0.751430,0.361504,0.575704,0.629803
structured_Beer,0.487113,0.968301,0.531915,0.948296,0.733906
structured_Fodors_Zagats,0.726378,0.954449,0.483139,0.972695,0.784165
structured_Amazon_Google,0.800970,0.700264,0.654146,0.804281,0.739915
